<a href="https://colab.research.google.com/github/SameekshaMenda/Second-Opinion-Model/blob/main/SecondMedicalModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### failed model


In [ ]:
!pip install pandas scikit-learn joblib torch transformers sentencepiece
!pip install easyocr opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 28.7 MB/s eta 0:00:00


In [ ]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 2s (119 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126666 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!pip install thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 66.2 MB/s eta 0:00:00


In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create a project folder to keep things organized
import os
project_path = '/content/drive/MyDrive/NextOpinion_Project'
if not os.path.exists(project_path):
    os.makedirs(project_path)
    print(f"Created project folder at: {project_path}")# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create a project folder to keep things organized
import os
project_path = '/content/drive/MyDrive/NextOpinion_Project'
if not os.path.exists(project_path):
    os.makedirs(project_path)
    print(f"Created project folder at: {project_path}")

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- Core Libraries ---
import cv2
import numpy as np
import pandas as pd
import torch
import re
import joblib
from PIL import Image
import os
from pdf2image import convert_from_path
from thefuzz import fuzz

# --- OCR and NER Libraries ---
import easyocr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, pipeline

# --- Machine Learning Libraries ---
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# ==============================================================================
#  CONFIGURATION: Define file paths for Google Drive
# ==============================================================================
BASE_PATH = '/content/drive/MyDrive/NextOpinion_Project/'
DATASET_PATH = os.path.join(BASE_PATH, 'Symptom-severity.csv') # Make sure this filename is correct!
MED_DB_PATH = os.path.join(BASE_PATH, 'medications_to_diseases.csv')
MODEL_PATH = os.path.join(BASE_PATH, 'disease_model.joblib')
ENCODER_PATH = os.path.join(BASE_PATH, 'label_encoder.joblib')
COLUMNS_PATH = os.path.join(BASE_PATH, 'symptom_columns.joblib')

# ==============================================================================
# PART 1: MACHINE LEARNING MODEL TRAINING
# ==============================================================================

# **UPDATED FUNCTION DEFINITION**
# ==============================================================================
# PART 1: MACHINE LEARNING MODEL TRAINING (FINAL REVISED VERSION)
# ==============================================================================

def train_disease_prediction_model(dataset_path):
    """
    Loads, cleans (fills NaNs, removes singletons), trains, and saves the model.
    """
    print("--- Starting ML Model Training ---")

    if not os.path.exists(MED_DB_PATH):
        print("Creating medication knowledge base...")
        # (code to create med_db.csv remains the same)
        med_data = {
            'MedicationName': ['Metoprolol', 'Lisinopril', 'Amlodipine', 'Atorvastatin', 'Metformin', 'Insulin', 'Paracetamol', 'Ibuprofen', 'Amoxicillin', 'Azithromycin'],
            'Disease': ['Hypertension', 'Hypertension', 'Hypertension', 'High Cholesterol', 'Diabetes', 'Diabetes', 'Fever', 'Pain', 'Bacterial Infection', 'Bacterial Infection']
        }
        pd.DataFrame(med_data).to_csv(MED_DB_PATH, index=False)


    try:
        df = pd.read_csv(dataset_path)
    except FileNotFoundError:
        print(f"ERROR: Dataset not found at '{dataset_path}'. Please check the filename.")
        return

    # ==========================================================================
    # ** NEW FIX: Fill any empty cells (NaNs) with 0 **
    # ==========================================================================
    # This assumes that a blank cell means the symptom is not present.
    df.fillna(0, inplace=True)
    print("Filled any empty cells (NaNs) with 0.")
    # ==========================================================================

    # This line helps clean up any hidden spaces around column names
    df.columns = df.columns.str.strip()

    # Define the correct column name for diseases
    disease_column = 'diseases' # From your screenshot

    # Filter out diseases that only appear once
    print(f"Original dataset size: {len(df)} rows")
    disease_counts = df[disease_column].value_counts()
    diseases_to_keep = disease_counts[disease_counts > 1].index.tolist()
    df_filtered = df[df[disease_column].isin(diseases_to_keep)]
    print(f"Filtered dataset size (removing singletons): {len(df_filtered)} rows")

    # Use the filtered DataFrame from now on
    X = df_filtered.drop(disease_column, axis=1)
    y = df_filtered[disease_column]

    # Convert all symptom columns to integer type, just in case
    X = X.astype(int)

    symptom_columns = X.columns.tolist()
    joblib.dump(symptom_columns, COLUMNS_PATH)

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    joblib.dump(label_encoder, ENCODER_PATH)

    # This will now work without error
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

    print("Training Logistic Regression model...")
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")

    joblib.dump(model, MODEL_PATH)
    print(f"--- Model and helper files saved successfully to '{BASE_PATH}' ---")


# ==============================================================================
# PART 2: OCR AND NER PIPELINE (Your Existing Code, Refined)
# ==============================================================================

def detect_text_with_easyocr_filtered(image_path, reader):
    print("Detecting text with EasyOCR...")
    results = reader.readtext(image_path)
    bounding_boxes = [np.array(bbox) for (bbox, text, confidence) in results if confidence > 0.3 and len(text.strip()) > 1]
    print(f"Found {len(bounding_boxes)} reliable text regions.")
    return bounding_boxes

def recognize_text_from_regions_enhanced(image, bounding_boxes, processor, model):
    print("Recognizing text with TrOCR...")
    recognized_texts = []
    image_rgb = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    for i, box in enumerate(bounding_boxes):
        try:
            x_min, y_min = np.min(box, axis=0).astype(int)
            x_max, y_max = np.max(box, axis=0).astype(int)
            padding = 10
            cropped_image = image_rgb.crop((max(0, x_min - padding), max(0, y_min - padding),
                                            min(image.shape[1], x_max + padding), min(image.shape[0], y_max + padding)))

            pixel_values = processor(images=cropped_image, return_tensors="pt").pixel_values
            generated_ids = model.generate(pixel_values, max_length=64)
            generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            cleaned_text = re.sub(r'[^\w\s\.\-]', '', generated_text.strip())
            if len(cleaned_text) > 1:
                recognized_texts.append(cleaned_text)
        except Exception as e:
            print(f"Error processing region {i}: {e}")
            continue

    full_text = " ".join(recognized_texts)
    print("Text recognition complete.")
    return full_text

def extract_medical_entities(text, ner_pipeline):
    print("Extracting medical entities with NER...")
    if not text.strip():
        return []
    ner_results = ner_pipeline(text, aggregation_strategy="simple")
    filtered_entities = [entity for entity in ner_results if entity['score'] > 0.8]
    print(f"Extracted {len(filtered_entities)} high-confidence entities.")
    return filtered_entities

# In Cell 3, replace the old classify_entities function with this one.
# In Cell 3, replace the old classify_entities function with this one.

def classify_entities(full_raw_text, entities, all_symptoms):
    """
    Uses a robust two-stage process to classify entities.
    1. A primary pass using high-confidence NER tags.
    2. A comprehensive "safety net" pass on the raw text to catch NER misses.
    """
    results = {
        "Medications": set(),
        "Symptoms": set(),
    }

    # --- Thresholds Tuned for Accuracy ---
    SYMPTOM_THRESHOLD = 90  # Stricter to prevent false positives
    MED_THRESHOLD = 88      # High confidence match for medications

    known_medications_lower = [
        'lisinopril', 'metoprolol', 'amlodipine', 'atorvastatin', 'metformin',
        'insulin', 'paracetamol', 'ibuprofen', 'amoxicillin', 'azithromycin'
    ]

    # --- STAGE 1: Primary Classification using NER Entities ---
    # This is fast and accurate for well-formed entities.
    for entity in entities:
        word_lower = entity['word'].lower().strip().replace('_', ' ')
        label_lower = entity['entity_group'].lower()

        if 'dis' in label_lower or 'symp' in label_lower:
            for symptom_in_list in all_symptoms:
                clean_symptom = symptom_in_list.lower().replace('_', ' ')
                if fuzz.ratio(clean_symptom, word_lower) > SYMPTOM_THRESHOLD:
                    results["Symptoms"].add(symptom_in_list)
                    break

    # --- STAGE 2: Comprehensive Safety Net on Raw Text ---
    # This is a fallback to catch anything the NER model missed entirely.
    # It is completely independent of the NER model's success.

    # Clean and split the entire OCR text into individual words
    raw_text_words = set(full_raw_text.lower().replace('_', ' ').split())

    for word in raw_text_words:
        # Safety check for symptoms
        for symptom_in_list in all_symptoms:
            clean_symptom = symptom_in_list.lower().replace('_', ' ')
            if fuzz.ratio(word, clean_symptom) > SYMPTOM_THRESHOLD:
                results["Symptoms"].add(symptom_in_list)

        # Safety check for medications
        for med in known_medications_lower:
            if fuzz.ratio(word, med) > MED_THRESHOLD:
                 results["Medications"].add(med.capitalize())

    # ==============================================================================

    results["Medications"] = list(results["Medications"])
    results["Symptoms"] = list(results["Symptoms"])
    return results

# ==============================================================================
# FINAL, MOST ROBUST VERSION of classify_entities
# ==============================================================================
# Make sure you have this import at the top of your script
from thefuzz import fuzz

# ==============================================================================
# FINAL, PRODUCTION-READY classify_entities FUNCTION (VERSION 6)
# ==============================================================================
def classify_entities(full_raw_text, entities, all_symptoms):
    """
    Uses a final, strict two-stage process to ensure maximum accuracy and
    eliminate false positives.
    """
    results = {
        "Medications": set(),
        "Symptoms": set(),
    }

    # Use a strict threshold. This demands a very close match.
    STRICT_THRESHOLD = 90
    MED_THRESHOLD = 88

    known_medications_lower = [
        'lisinopril', 'metoprolol', 'amlodipine', 'atorvastatin', 'metformin',
        'insulin', 'paracetamol', 'ibuprofen', 'amoxicillin', 'azithromycin'
    ]

    # --- STAGE 1: Primary Pass (No Change) ---
    # This is a quick pass for anything the NER model identifies perfectly.
    for entity in entities:
        word_lower = entity['word'].lower().strip().replace('_', ' ')
        label_lower = entity['entity_group'].lower()
        if 'dis' in label_lower or 'symp' in label_lower:
            for symptom_in_list in all_symptoms:
                clean_symptom = symptom_in_list.lower().replace('_', ' ')
                if fuzz.ratio(clean_symptom, word_lower) > STRICT_THRESHOLD:
                    results["Symptoms"].add(symptom_in_list)
                    break

    # --- STAGE 2: Strict Safety Net ---
    # Generates phrases from the raw text and compares them strictly.
    words = full_raw_text.lower().replace('_', ' ').split()
    phrases = set(words) # Start with single words
    for i in range(len(words) - 1):
        phrases.add(words[i] + " " + words[i+1]) # Add two-word phrases

    for phrase in phrases:
        # Symptom Check: Use the strict fuzz.ratio
        for symptom_in_list in all_symptoms:
            clean_symptom = symptom_in_list.lower().replace('_', ' ')
            if fuzz.ratio(phrase, clean_symptom) > STRICT_THRESHOLD:
                results["Symptoms"].add(symptom_in_list)

    # Medication Check: This proven logic remains unchanged.
    for word in words:
        for med in known_medications_lower:
            if fuzz.ratio(word, med) > MED_THRESHOLD:
                 results["Medications"].add(med.capitalize())

    results["Medications"] = list(results["Medications"])
    results["Symptoms"] = list(results["Symptoms"])
    return results

# ==============================================================================
# PART 3: DISEASE PREDICTION ENGINE
# ==============================================================================

def predict_disease_from_symptoms(symptoms, model, label_encoder, symptom_columns):
    """
    Predicts disease using the pre-trained ML model.
    """
    print("Predicting disease from symptoms using ML model...")
    if not symptoms:
        return "No relevant symptoms found to make a prediction.", 0.0

    # Create an input vector of all zeros
    input_vector = pd.DataFrame(columns=symptom_columns)
    input_vector.loc[0] = 0

    # Set the columns for the extracted symptoms to 1
    found_symptoms_count = 0
    for symptom in symptoms:
        if symptom in symptom_columns:
            input_vector[symptom] = 1
            found_symptoms_count += 1
            print(f"- Found matching symptom in dataset: {symptom}")

    if found_symptoms_count == 0:
        return "Extracted symptoms do not match the knowledge base.", 0.0

    # Make prediction
    prediction_encoded = model.predict(input_vector)[0]
    prediction_proba = model.predict_proba(input_vector)

    # Decode the prediction to the actual disease name
    disease_name = label_encoder.inverse_transform([prediction_encoded])[0]
    confidence = np.max(prediction_proba) * 100

    return disease_name, confidence

def get_evidence_from_medications(medications, med_db):
    """
    Finds potential diseases based on medications from a knowledge base.
    """
    if med_db is None or not medications:
        return {}

    evidence = {}
    for med in medications:
        matched_diseases = med_db[med_db['MedicationName'].str.lower() == med.lower()]['Disease']
        for disease in matched_diseases:
            if disease not in evidence:
                evidence[disease] = []
            evidence[disease].append(med)

    return evidence


# ==============================================================================
# PART 4: MAIN EXECUTION WORKFLOW
# ==============================================================================

if __name__ == '__main__':
    # --------------------------------------------------------------------------
    # STEP 1: Run this line ONCE to train and save the model
    # --------------------------------------------------------------------------
    # train_disease_prediction_model(DATASET_PATH)
    # --------------------------------------------------------------------------

    # --------------------------------------------------------------------------
    # STEP 2: Run predictions using the saved model
    # --------------------------------------------------------------------------
    PDF_PATH = 'cardiology_report_sample.pdf' # <-- The PDF you want to analyze
    IMAGE_PATH = 'temp_converted_image.jpg' # A temporary file for the image

    try:
        # --- NEW: PDF to Image Conversion Step ---
        print(f"Converting PDF '{PDF_PATH}' to an image...")
        # Convert the first page of the PDF to a PIL Image
        images = convert_from_path(PDF_PATH)
        if not images:
            raise Exception("PDF is empty or could not be converted.")

        # Save the first page as a temporary JPG file
        images[0].save(IMAGE_PATH, 'JPEG')
        print("PDF converted successfully.\n")
        # --- End of New Step ---


        # --- Load all models and databases ---
        print("Loading all required models and data...")
        # ML Models
        disease_model = joblib.load(MODEL_PATH)
        label_encoder = joblib.load(ENCODER_PATH)
        symptom_columns = joblib.load(COLUMNS_PATH)
        # Knowledge Base
        med_db = pd.read_csv(MED_DB_PATH)
        # OCR/NER Models
        ocr_reader = easyocr.Reader(['en'])
        trocr_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
        trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
        ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all")
        print("All models loaded successfully.\n")

        # --- Full Analysis Pipeline (The rest of the code is unchanged) ---
        # 1. Load Image
        image = cv2.imread(IMAGE_PATH)
        if image is None:
            raise FileNotFoundError(f"Image not found at {IMAGE_PATH}")

        # 2. OCR and Text Recognition
        bounding_boxes = detect_text_with_easyocr_filtered(IMAGE_PATH, ocr_reader)
        full_text = recognize_text_from_regions_enhanced(image, bounding_boxes, trocr_processor, trocr_model)

        print("\n--- Raw Recognized Text ---")
        print(full_text)

        # 3. NER and Entity Classification
        if full_text.strip():
            medical_entities = extract_medical_entities(full_text, ner_pipeline)
            classified_info = classify_entities(full_text, medical_entities, symptom_columns)

            extracted_medications = classified_info["Medications"]
            extracted_symptoms = classified_info["Symptoms"]

            print("\n--- Extracted Medical Information ---")
            print(f"Identified Medications: {', '.join(extracted_medications) if extracted_medications else 'None'}")
            print(f"Identified Symptoms: {', '.join(extracted_symptoms) if extracted_symptoms else 'None'}")

            # 4. Disease Prediction
            predicted_disease, confidence = predict_disease_from_symptoms(
                extracted_symptoms, disease_model, label_encoder, symptom_columns
            )

            medication_evidence = get_evidence_from_medications(extracted_medications, med_db)

            # 5. Display Final Results
            print("\n" + "="*60)
            print("         FINAL ANALYSIS REPORT")
            print("="*60)
            print(f"\n>> PREDICTED CONDITION (from symptoms): {predicted_disease} ({confidence:.2f}% confidence)")

            if medication_evidence:
                print("\n>> SUPPORTING EVIDENCE (from medications):")
                for disease, meds in medication_evidence.items():
                    print(f"   - The medication(s) {', '.join(meds)} are commonly prescribed for '{disease}'.")

            if not extracted_symptoms and not extracted_medications:
                 print("\nCould not extract sufficient medical information from the document.")

        else:
            print("No text was recognized from the image.")

    except FileNotFoundError as e:
        print(f"\nERROR: A required file was not found. Please ensure all model files and data files exist.")
        print(f"Details: {e}")
        print("If this is your first run, please remember to run the training step first.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Converting PDF 'cardiology_report_sample.pdf' to an image...
PDF converted successfully.

Loading all required models and data...


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


All models loaded successfully.

Detecting text with EasyOCR...
Found 77 reliable text regions.
Recognizing text with TrOCR...
Text recognition complete.

--- Raw Recognized Text ---
RIVERBEND HEART INSTITUTE CARDIOLOGY DIAGNOSTIC REPORT PATIENT NAME JOHN DOE REPORT DATE 01 OCT 2025 MEDICAL RECORD NO. RB-2025-000123 REPORT TIME 1030 DATE OF BIRTH 1972-04-15 53 YEARS SEX MALE REFERRING PHYSICIAN DR. A. K. SHARMA GENERAL MEDICINE REPORT ID CR-2025-1001 CLINICAL SUMMARY HISTORY PAIGHT PRESENTED WITH PROFESSIVE EXEPTOTAL OYSPARA OVER 3 MONTHS ONTRONES AND TWO EXFOODES OF NEAR-SYNCOPE. HISTORY OF HYPERTENSION FOR 12 YEARS AND SMOKER STOPPED 5 YEARS AGO. RECENT EXEMTOTAL CHES DISCOUNT PRINTED. REFERED TO CARDBOOK EVALUATION AND INVESTATION INCLUDING ECG CHEST X-RAY ECHOCARDIGRAM AND CARDIAC BIOMARKERS. VITAL SIGNS  INVESTIGATIONS BLOOD PRESSURE 15092 MMHG HEART RATE 88 BPM RESPIRATORY RATE 18 MIN TEMPERATURE 36.8 C SPO2 ROOM AIR 95  TEST RESULT REFERENCE TROPONIN I 0.08 MGML  0.04 MGML BNP 9

In [ ]:
# --- Core Libraries ---
import cv2
import numpy as np
import pandas as pd
import torch
import re
import joblib
from PIL import Image
import os
from pdf2image import convert_from_path
from thefuzz import fuzz

# --- OCR and NER Libraries ---
import easyocr
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, pipeline

# --- Machine Learning Libraries ---
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# ==============================================================================
#  CONFIGURATION: Define file paths for Google Drive
# ==============================================================================
BASE_PATH = '/content/drive/MyDrive/NextOpinion_Project/'
DATASET_PATH = os.path.join(BASE_PATH, 'Symptom-severity.csv')
MED_DB_PATH = os.path.join(BASE_PATH, 'medications_to_diseases.csv')
MODEL_PATH = os.path.join(BASE_PATH, 'disease_model.joblib')
ENCODER_PATH = os.path.join(BASE_PATH, 'label_encoder.joblib')
COLUMNS_PATH = os.path.join(BASE_PATH, 'symptom_columns.joblib')

# ==============================================================================
# PART 1: MACHINE LEARNING MODEL TRAINING
# ==============================================================================

def train_disease_prediction_model(dataset_path):
    """
    Loads, cleans (fills NaNs, removes singletons), trains, and saves the model.
    """
    print("--- Starting ML Model Training ---")

    if not os.path.exists(MED_DB_PATH):
        print("Creating medication knowledge base...")
        med_data = {
            'MedicationName': ['Metoprolol', 'Lisinopril', 'Amlodipine', 'Atorvastatin', 'Furosemide',
                             'Spironolactone', 'Digoxin', 'Warfarin', 'Aspirin', 'Clopidogrel',
                             'Metformin', 'Insulin', 'Paracetamol', 'Ibuprofen', 'Amoxicillin'],
            'Disease': ['Hypertension', 'Hypertension', 'Hypertension', 'High Cholesterol', 'Heart Failure',
                       'Heart Failure', 'Heart Failure', 'Atrial Fibrillation', 'Coronary Artery Disease',
                       'Coronary Artery Disease', 'Diabetes', 'Diabetes', 'Fever', 'Pain', 'Bacterial Infection']
        }
        pd.DataFrame(med_data).to_csv(MED_DB_PATH, index=False)

    try:
        df = pd.read_csv(dataset_path)
    except FileNotFoundError:
        print(f"ERROR: Dataset not found at '{dataset_path}'. Please check the filename.")
        return

    # Fill any empty cells (NaNs) with 0
    df.fillna(0, inplace=True)
    print("Filled any empty cells (NaNs) with 0.")

    # Clean up column names
    df.columns = df.columns.str.strip()

    # Define the correct column name for diseases
    disease_column = 'diseases'

    # Filter out diseases that only appear once
    print(f"Original dataset size: {len(df)} rows")
    disease_counts = df[disease_column].value_counts()
    diseases_to_keep = disease_counts[disease_counts > 1].index.tolist()
    df_filtered = df[df[disease_column].isin(diseases_to_keep)]
    print(f"Filtered dataset size (removing singletons): {len(df_filtered)} rows")

    # Use the filtered DataFrame from now on
    X = df_filtered.drop(disease_column, axis=1)
    y = df_filtered[disease_column]

    # Convert all symptom columns to integer type
    X = X.astype(int)

    symptom_columns = X.columns.tolist()
    joblib.dump(symptom_columns, COLUMNS_PATH)

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    joblib.dump(label_encoder, ENCODER_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

    print("Training Logistic Regression model...")
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")

    joblib.dump(model, MODEL_PATH)
    print(f"--- Model and helper files saved successfully to '{BASE_PATH}' ---")

# ==============================================================================
# PART 2: IMPROVED OCR AND NER PIPELINE
# ==============================================================================

def detect_text_with_easyocr_filtered(image_path, reader):
    print("Detecting text with EasyOCR...")
    results = reader.readtext(image_path)
    bounding_boxes = [np.array(bbox) for (bbox, text, confidence) in results if confidence > 0.3 and len(text.strip()) > 1]
    print(f"Found {len(bounding_boxes)} reliable text regions.")
    return bounding_boxes

def recognize_text_from_regions_enhanced(image, bounding_boxes, processor, model):
    print("Recognizing text with TrOCR...")
    recognized_texts = []
    image_rgb = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    for i, box in enumerate(bounding_boxes):
        try:
            x_min, y_min = np.min(box, axis=0).astype(int)
            x_max, y_max = np.max(box, axis=0).astype(int)
            padding = 10
            cropped_image = image_rgb.crop((max(0, x_min - padding), max(0, y_min - padding),
                                            min(image.shape[1], x_max + padding), min(image.shape[0], y_max + padding)))

            pixel_values = processor(images=cropped_image, return_tensors="pt").pixel_values
            generated_ids = model.generate(pixel_values, max_length=64)
            generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            cleaned_text = re.sub(r'[^\w\s\.\-]', '', generated_text.strip())
            if len(cleaned_text) > 1:
                recognized_texts.append(cleaned_text)
        except Exception as e:
            print(f"Error processing region {i}: {e}")
            continue

    full_text = " ".join(recognized_texts)
    print("Text recognition complete.")
    return full_text

def preprocess_medical_text(text):
    """
    Clean and normalize medical text for better entity recognition
    """
    # Common OCR corrections for medical terms
    corrections = {
        'oyspara': 'dyspnea', 'dyspara': 'dyspnea', 'oy spara': 'dyspnea',
        'onthrones': 'orthopnea', 'ontrones': 'orthopnea', 'orthop nea': 'orthopnea',
        'exeptotal': 'exertional', 'exemtotal': 'exertional', 'exertion al': 'exertional',
        'syn cope': 'syncope', 'synco pe': 'syncope', 'near syncope': 'near-syncope',
        'hypertrophy': 'hypertrophy', 'hypertroph y': 'hypertrophy',
        'hypokinesia': 'hypokinesia', 'hypokinesi a': 'hypokinesia',
        'dysfunction': 'dysfunction', 'dysfuncti on': 'dysfunction',
        'regurgitation': 'regurgitation', 'regurgitatio n': 'regurgitation',
        'stenosis': 'stenosis', 'stenos is': 'stenosis',
        'cardiomyopathy': 'cardiomyopathy', 'cardiomyop athy': 'cardiomyopathy',
        'hypertension': 'hypertension', 'hyperten sion': 'hypertension',
        'troponin': 'troponin', 'tropon in': 'troponin',
        'creatinine': 'creatinine', 'creatin ine': 'creatinine',
        'hemoglobin': 'hemoglobin', 'hemog lobin': 'hemoglobin',
        'ventricular': 'ventricular', 'ventric ular': 'ventricular',
        'ejection': 'ejection', 'ejec tion': 'ejection'
    }

    text_lower = text.lower()
    for wrong, correct in corrections.items():
        text_lower = text_lower.replace(wrong, correct)

    return text_lower

def extract_medical_entities(text, ner_pipeline):
    print("Extracting medical entities with NER...")
    if not text.strip():
        return []
    ner_results = ner_pipeline(text, aggregation_strategy="simple")
    filtered_entities = [entity for entity in ner_results if entity['score'] > 0.8]
    print(f"Extracted {len(filtered_entities)} high-confidence entities.")
    return filtered_entities

def improved_classify_entities(full_raw_text, entities, all_symptoms):
    """
    Improved entity classification with better medical term handling
    """
    results = {
        "Medications": set(),
        "Symptoms": set(),
        "Conditions": set(),
        "Findings": set()
    }

    # Preprocess the text
    cleaned_text = preprocess_medical_text(full_raw_text)

    # Expanded medical terminology
    cardiac_symptoms = [
        'dyspnea', 'orthopnea', 'syncope', 'chest pain', 'chest discomfort',
        'fatigue', 'palpitations', 'edema', 'shortness of breath', 'near-syncope',
        'exertional dyspnea', 'progressive dyspnea'
    ]

    cardiac_conditions = [
        'hypertension', 'cardiomyopathy', 'heart failure', 'ventricular dysfunction',
        'mitral regurgitation', 'aortic stenosis', 'coronary artery disease',
        'arrhythmia', 'tachycardia', 'bradycardia', 'left ventricular hypertrophy',
        'systolic dysfunction', 'ischemic cardiomyopathy', 'dilated cardiomyopathy'
    ]

    cardiac_findings = [
        'ejection fraction', 'lvef', 'hypokinesia', 'global hypokinesia',
        'ventricular dilation', 'left atrial enlargement', 'st-t changes',
        'pvc', 'premature ventricular complexes'
    ]

    known_medications = [
        'lisinopril', 'metoprolol', 'amlodipine', 'atorvastatin', 'furosemide',
        'spironolactone', 'digoxin', 'warfarin', 'aspirin', 'clopidogrel',
        'metformin', 'insulin'
    ]

    # Combine all medical terms for matching
    all_medical_terms = cardiac_symptoms + cardiac_conditions + cardiac_findings + known_medications

    # Direct string matching for known medical terms
    words = cleaned_text.split()

    # Check for single words
    for word in words:
        for term in all_medical_terms:
            if fuzz.ratio(word, term) > 85:  # Slightly lower threshold for better recall
                if term in cardiac_symptoms:
                    results["Symptoms"].add(term)
                elif term in cardiac_conditions:
                    results["Conditions"].add(term)
                elif term in cardiac_findings:
                    results["Findings"].add(term)
                elif term in known_medications:
                    results["Medications"].add(term.capitalize())

    # Check for multi-word phrases (2 words)
    for i in range(len(words) - 1):
        two_word_phrase = f"{words[i]} {words[i+1]}"
        for term in all_medical_terms:
            if ' ' in term and fuzz.ratio(two_word_phrase, term) > 85:
                if term in cardiac_symptoms:
                    results["Symptoms"].add(term)
                elif term in cardiac_conditions:
                    results["Conditions"].add(term)
                elif term in cardiac_findings:
                    results["Findings"].add(term)

    # Check for multi-word phrases (3 words)
    for i in range(len(words) - 2):
        three_word_phrase = f"{words[i]} {words[i+1]} {words[i+2]}"
        for term in all_medical_terms:
            if three_word_phrase.count(' ') == 2 and fuzz.ratio(three_word_phrase, term) > 85:
                if term in cardiac_symptoms:
                    results["Symptoms"].add(term)
                elif term in cardiac_conditions:
                    results["Conditions"].add(term)
                elif term in cardiac_findings:
                    results["Findings"].add(term)

    # Remove duplicates and convert to lists
    results["Medications"] = list(results["Medications"])
    results["Symptoms"] = list(results["Symptoms"])
    results["Conditions"] = list(results["Conditions"])
    results["Findings"] = list(results["Findings"])

    return results

# ==============================================================================
# PART 3: IMPROVED DISEASE PREDICTION ENGINE
# ==============================================================================

def improved_predict_disease(symptoms, conditions, model, label_encoder, symptom_columns):
    """
    Enhanced prediction that considers both symptoms and conditions
    """
    print("Running improved disease prediction...")

    if not symptoms and not conditions:
        return "Insufficient information for prediction", 0.0

    # Create input vector
    input_vector = pd.DataFrame(columns=symptom_columns)
    input_vector.loc[0] = 0

    # Map symptoms and conditions to dataset columns
    found_count = 0
    all_findings = symptoms + conditions

    for finding in all_findings:
        # Try direct match first
        if finding in symptom_columns:
            input_vector[finding] = 1
            found_count += 1
            print(f"- Found matching finding: {finding}")
        else:
            # Try fuzzy matching for close matches
            for column in symptom_columns:
                if fuzz.ratio(finding.lower(), column.lower()) > 80:
                    input_vector[column] = 1
                    found_count += 1
                    print(f"- Found similar finding: {finding} -> {column}")
                    break

    if found_count == 0:
        return "No matching symptoms in knowledge base", 0.0

    try:
        # Make prediction
        prediction_encoded = model.predict(input_vector)[0]
        prediction_proba = model.predict_proba(input_vector)

        disease_name = label_encoder.inverse_transform([prediction_encoded])[0]
        confidence = np.max(prediction_proba) * 100

        return disease_name, confidence
    except Exception as e:
        return f"Prediction error: {str(e)}", 0.0

def get_evidence_from_medications(medications, med_db):
    """
    Finds potential diseases based on medications from a knowledge base.
    """
    if med_db is None or not medications:
        return {}

    evidence = {}
    for med in medications:
        matched_diseases = med_db[med_db['MedicationName'].str.lower() == med.lower()]['Disease']
        for disease in matched_diseases:
            if disease not in evidence:
                evidence[disease] = []
            evidence[disease].append(med)

    return evidence

def analyze_report_context(full_text, classified_info):
    """
    Analyze the report context for additional insights
    """
    context_findings = []
    text_lower = full_text.lower()

    # Check for key cardiac indicators
    if "ejection fraction" in text_lower and "30" in text_lower:
        context_findings.append("Severely reduced ejection fraction (30%) - indicates severe systolic dysfunction")

    if "ventricular" in text_lower and "dilation" in text_lower:
        context_findings.append("Ventricular dilation noted")

    if "left atrial enlargement" in text_lower:
        context_findings.append("Left atrial enlargement present")

    if "hypertension" in text_lower:
        context_findings.append("History of hypertension")

    if "global hypokinesia" in text_lower:
        context_findings.append("Global hypokinesia - reduced contractility throughout the ventricle")

    if "st-t changes" in text_lower:
        context_findings.append("ST-T changes on ECG - may indicate ischemia or strain")

    # Add findings from entity classification
    if classified_info["Findings"]:
        context_findings.extend([f"Finding: {finding}" for finding in classified_info["Findings"]])

    return context_findings

# ==============================================================================
# PART 4: MAIN EXECUTION WORKFLOW
# ==============================================================================

def analyze_cardiology_report(pdf_path):
    """
    Main function to analyze cardiology reports
    """
    try:
        # Convert PDF to image
        print(f"Converting PDF '{pdf_path}' to image...")
        images = convert_from_path(pdf_path)
        if not images:
            raise Exception("PDF conversion failed")

        image_path = 'temp_cardio_image.jpg'
        images[0].save(image_path, 'JPEG')
        print("PDF converted successfully.\n")

        # Load models
        print("Loading models...")
        disease_model = joblib.load(MODEL_PATH)
        label_encoder = joblib.load(ENCODER_PATH)
        symptom_columns = joblib.load(COLUMNS_PATH)
        med_db = pd.read_csv(MED_DB_PATH)

        ocr_reader = easyocr.Reader(['en'])
        trocr_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
        trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
        ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all")
        print("Models loaded successfully.\n")

        # OCR processing
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found at {image_path}")

        bounding_boxes = detect_text_with_easyocr_filtered(image_path, ocr_reader)
        full_text = recognize_text_from_regions_enhanced(image, bounding_boxes, trocr_processor, trocr_model)

        print("\n--- Raw Recognized Text ---")
        print(full_text)

        if full_text.strip():
            # Use improved entity classification
            medical_entities = extract_medical_entities(full_text, ner_pipeline)
            classified_info = improved_classify_entities(full_text, medical_entities, symptom_columns)

            extracted_medications = classified_info["Medications"]
            extracted_symptoms = classified_info["Symptoms"]
            extracted_conditions = classified_info["Conditions"]
            extracted_findings = classified_info["Findings"]

            print("\n--- Improved Medical Information Extraction ---")
            print(f"Medications: {', '.join(extracted_medications) if extracted_medications else 'None'}")
            print(f"Symptoms: {', '.join(extracted_symptoms) if extracted_symptoms else 'None'}")
            print(f"Conditions: {', '.join(extracted_conditions) if extracted_conditions else 'None'}")
            print(f"Findings: {', '.join(extracted_findings) if extracted_findings else 'None'}")

            # Improved prediction
            predicted_disease, confidence = improved_predict_disease(
                extracted_symptoms, extracted_conditions, disease_model, label_encoder, symptom_columns
            )

            medication_evidence = get_evidence_from_medications(extracted_medications, med_db)
            context_findings = analyze_report_context(full_text, classified_info)

            # Display results
            print("\n" + "="*70)
            print("               COMPREHENSIVE CARDIOLOGY ANALYSIS REPORT")
            print("="*70)

            if isinstance(predicted_disease, str) and "error" not in predicted_disease.lower() and "insufficient" not in predicted_disease.lower():
                print(f"\n>> PREDICTED CONDITION: {predicted_disease} ({confidence:.2f}% confidence)")
            else:
                print(f"\n>> PREDICTION: {predicted_disease}")

            # Display contextual findings
            if context_findings:
                print("\n>> CLINICAL CONTEXT:")
                for finding in context_findings:
                    print(f"   • {finding}")

            # Display medication evidence
            if medication_evidence:
                print("\n>> MEDICATION EVIDENCE:")
                for disease, meds in medication_evidence.items():
                    print(f"   • {', '.join(meds)} → commonly used for {disease}")

            # Provide clinical interpretation
            print("\n>> CLINICAL INTERPRETATION:")
            if any(symptom in ['dyspnea', 'orthopnea', 'syncope'] for symptom in extracted_symptoms):
                print("   • Symptoms suggest significant cardiac compromise")
            if any(condition in ['cardiomyopathy', 'heart failure', 'ventricular dysfunction'] for condition in extracted_conditions):
                print("   • Evidence of structural heart disease present")
            if extracted_medications:
                print("   • Medication profile supports cardiac diagnosis")

            if not extracted_symptoms and not extracted_conditions and not extracted_medications:
                print("\n>> NOTE: Limited medical information extracted. Manual review recommended.")

        else:
            print("No text recognized from document.")

        # Clean up temporary file
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"\nCleaned up temporary file: {image_path}")

    except FileNotFoundError as e:
        print(f"\nERROR: Required file not found: {e}")
        print("Please ensure all model files exist and you've run the training step first.")
    except Exception as e:
        print(f"\nAnalysis error: {e}")

# ==============================================================================
# EXECUTION BLOCK
# ==============================================================================

if __name__ == '__main__':
    """
    Main execution block
    """
    print("CARDIOLOGY REPORT ANALYSIS SYSTEM")
    print("=" * 50)

    # Step 1: Train the model (run this once initially)
    # Uncomment the line below if you need to train the model first
    # train_disease_prediction_model(DATASET_PATH)

    # Step 2: Analyze the cardiology report
    pdf_file = 'cardiology_report_sample.pdf'  # Change this to your PDF file name

    if os.path.exists(pdf_file):
        analyze_cardiology_report(pdf_file)
    else:
        print(f"PDF file '{pdf_file}' not found.")
        print("Please ensure the PDF file is in the current directory.")

    print("\n" + "=" * 50)
    print("Analysis complete!")

CARDIOLOGY REPORT ANALYSIS SYSTEM
Converting PDF 'cardiology_report_sample.pdf' to image...
PDF converted successfully.

Loading models...


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


Models loaded successfully.

Detecting text with EasyOCR...
Found 77 reliable text regions.
Recognizing text with TrOCR...
Text recognition complete.

--- Raw Recognized Text ---
RIVERBEND HEART INSTITUTE CARDIOLOGY DIAGNOSTIC REPORT PATIENT NAME JOHN DOE REPORT DATE 01 OCT 2025 MEDICAL RECORD NO. RB-2025-000123 REPORT TIME 1030 DATE OF BIRTH 1972-04-15 53 YEARS SEX MALE REFERRING PHYSICIAN DR. A. K. SHARMA GENERAL MEDICINE REPORT ID CR-2025-1001 CLINICAL SUMMARY HISTORY PAIGHT PRESENTED WITH PROFESSIVE EXEPTOTAL OYSPARA OVER 3 MONTHS ONTRONES AND TWO EXFOODES OF NEAR-SYNCOPE. HISTORY OF HYPERTENSION FOR 12 YEARS AND SMOKER STOPPED 5 YEARS AGO. RECENT EXEMTOTAL CHES DISCOUNT PRINTED. REFERED TO CARDBOOK EVALUATION AND INVESTATION INCLUDING ECG CHEST X-RAY ECHOCARDIGRAM AND CARDIAC BIOMARKERS. VITAL SIGNS  INVESTIGATIONS BLOOD PRESSURE 15092 MMHG HEART RATE 88 BPM RESPIRATORY RATE 18 MIN TEMPERATURE 36.8 C SPO2 ROOM AIR 95  TEST RESULT REFERENCE TROPONIN I 0.08 MGML  0.04 MGML BNP 950 P

In [ ]:
# Add this import at the top
import google.generativeai as genai

# ==============================================================================
# GEMINI LLM ANALYSIS COMPONENT
# ==============================================================================

def setup_gemini(api_key=None):
    """
    Setup Gemini API
    """
    try:
        if api_key:
            genai.configure(api_key=api_key)
        else:
            # Try to use from environment variable or default
            genai.configure()

        model = genai.GenerativeModel('gemini-1.5-flash')
        return model
    except Exception as e:
        print(f"Gemini setup failed: {e}")
        return None

def get_gemini_analysis(extracted_data, clinical_context):
    """
    Get comprehensive analysis from Gemini LLM
    """
    print("🔄 Consulting Gemini LLM for clinical analysis...")

    prompt = f"""
    ACT AS AN EXPERIENCED CARDIOLOGIST. Analyze this clinical case:

    EXTRACTED MEDICAL INFORMATION:
    - Symptoms: {extracted_data['symptoms']}
    - Conditions: {extracted_data['conditions']}
    - Findings: {extracted_data['findings']}
    - Medications: {extracted_data['medications']}

    CLINICAL CONTEXT:
    {clinical_context}

    PROVIDE THIS EXACT FORMAT:
    DIAGNOSIS: [Your primary diagnosis]
    CONFIDENCE: [Low/Medium/High/Very High as percentage 0-100%]
    EVIDENCE: [Bullet points of supporting evidence]
    DIFFERENTIALS: [Other possible diagnoses]
    RECOMMENDATIONS: [Next steps]

    Be concise and clinical in your assessment.
    """

    try:
        model = setup_gemini()
        if not model:
            return {
                "diagnosis": "Gemini unavailable",
                "confidence": 0,
                "evidence": ["LLM service not configured"],
                "reasoning": "Please set up Gemini API key"
            }

        response = model.generate_content(prompt)

        if response.text:
            return parse_structured_gemini_response(response.text)
        else:
            return {
                "diagnosis": "Analysis failed",
                "confidence": 0,
                "evidence": ["No response from LLM"],
                "reasoning": "Empty response from Gemini"
            }

    except Exception as e:
        print(f"Gemini analysis error: {e}")
        return {
            "diagnosis": f"LLM error: {str(e)}",
            "confidence": 0,
            "evidence": ["Analysis failed"],
            "reasoning": "Exception occurred"
        }

def parse_structured_gemini_response(response_text):
    """
    Parse Gemini response into structured format
    """
    try:
        result = {
            "diagnosis": "Unknown",
            "confidence": 50,
            "evidence": [],
            "reasoning": response_text[:500] + "..." if len(response_text) > 500 else response_text
        }

        lines = response_text.split('\n')
        current_section = None

        for line in lines:
            line = line.strip()
            if not line:
                continue

            if line.startswith('DIAGNOSIS:'):
                result["diagnosis"] = line.replace('DIAGNOSIS:', '').strip()
            elif line.startswith('CONFIDENCE:'):
                conf_text = line.replace('CONFIDENCE:', '').strip()
                # Extract percentage
                if '%' in conf_text:
                    conf_num = ''.join(filter(str.isdigit, conf_text.split('%')[0]))
                    if conf_num:
                        result["confidence"] = min(100, max(0, int(conf_num)))
                else:
                    # Handle Low/Medium/High
                    conf_lower = conf_text.lower()
                    if 'very high' in conf_lower or 'high' in conf_lower:
                        result["confidence"] = 80
                    elif 'medium' in conf_lower:
                        result["confidence"] = 60
                    elif 'low' in conf_lower:
                        result["confidence"] = 40
            elif line.startswith('EVIDENCE:'):
                current_section = 'evidence'
            elif line.startswith('DIFFERENTIALS:'):
                current_section = 'differentials'
            elif line.startswith('RECOMMENDATIONS:'):
                current_section = 'recommendations'
            elif current_section == 'evidence' and (line.startswith('-') or line.startswith('•')):
                result["evidence"].append(line.lstrip('-•').strip())

        return result

    except Exception as e:
        print(f"Error parsing Gemini response: {e}")
        return {
            "diagnosis": "Parse error",
            "confidence": 50,
            "evidence": ["Could not parse LLM response"],
            "reasoning": response_text[:300] if response_text else "No response"
        }

# ==============================================================================
# ENHANCED MAIN FUNCTION WITH DUAL ANALYSIS
# ==============================================================================

def analyze_with_dual_systems(pdf_path, gemini_api_key=None):
    """
    Run both ML model and Gemini LLM, then compare results
    """
    try:
        # Convert PDF to image (your existing code)
        print(f"Converting PDF '{pdf_path}' to image...")
        images = convert_from_path(pdf_path)
        if not images:
            raise Exception("PDF conversion failed")

        image_path = 'temp_cardio_image.jpg'
        images[0].save(image_path, 'JPEG')
        print("PDF converted successfully.\n")

        # Load models (your existing code)
        print("Loading ML models...")
        disease_model = joblib.load(MODEL_PATH)
        label_encoder = joblib.load(ENCODER_PATH)
        symptom_columns = joblib.load(COLUMNS_PATH)
        med_db = pd.read_csv(MED_DB_PATH)

        ocr_reader = easyocr.Reader(['en'])
        trocr_processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
        trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')
        ner_pipeline = pipeline("ner", model="d4data/biomedical-ner-all")
        print("ML models loaded successfully.\n")

        # OCR processing (your existing code)
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found at {image_path}")

        bounding_boxes = detect_text_with_easyocr_filtered(image_path, ocr_reader)
        full_text = recognize_text_from_regions_enhanced(image, bounding_boxes, trocr_processor, trocr_model)

        print("\n--- Raw Recognized Text ---")
        print(full_text[:500] + "..." if len(full_text) > 500 else full_text)

        if not full_text.strip():
            print("No text recognized from document.")
            return

        # Entity extraction (your existing code)
        medical_entities = extract_medical_entities(full_text, ner_pipeline)
        classified_info = improved_classify_entities(full_text, medical_entities, symptom_columns)

        extracted_medications = classified_info["Medications"]
        extracted_symptoms = classified_info["Symptoms"]
        extracted_conditions = classified_info["Conditions"]
        extracted_findings = classified_info["Findings"]

        print("\n--- Medical Information Extraction ---")
        print(f"Medications: {', '.join(extracted_medications) if extracted_medications else 'None'}")
        print(f"Symptoms: {', '.join(extracted_symptoms) if extracted_symptoms else 'None'}")
        print(f"Conditions: {', '.join(extracted_conditions) if extracted_conditions else 'None'}")
        print(f"Findings: {', '.join(extracted_findings) if extracted_findings else 'None'}")

        # ==========================================================================
        # DUAL ANALYSIS: ML MODEL + GEMINI LLM
        # ==========================================================================

        print("\n" + "="*70)
        print("               DUAL ANALYSIS: ML MODEL + GEMINI LLM")
        print("="*70)

        # ANALYSIS 1: ML MODEL (Your original system)
        print("\n🔬 ANALYSIS 1: MACHINE LEARNING MODEL")
        print("-" * 40)

        ml_diagnosis, ml_confidence, ml_mappings = adaptive_predict_disease(
            extracted_symptoms, extracted_conditions, extracted_findings,
            disease_model, label_encoder, symptom_columns
        )

        print(f"🏥 DIAGNOSIS: {ml_diagnosis}")
        print(f"📊 CONFIDENCE: {ml_confidence:.1f}%")

        if ml_mappings:
            print(f"🔍 MAPPED SYMPTOMS: {len(ml_mappings)} found")
            for mapping in ml_mappings[:3]:  # Show top 3
                print(f"   • {mapping}")

        # ANALYSIS 2: GEMINI LLM
        print("\n🤖 ANALYSIS 2: GEMINI LLM CLINICAL REASONING")
        print("-" * 40)

        extracted_data = {
            'symptoms': extracted_symptoms,
            'conditions': extracted_conditions,
            'findings': extracted_findings,
            'medications': extracted_medications
        }

        clinical_context = create_clinical_context(full_text, classified_info)
        gemini_result = get_gemini_analysis(extracted_data, clinical_context)

        print(f"🏥 DIAGNOSIS: {gemini_result['diagnosis']}")
        print(f"📊 CONFIDENCE: {gemini_result['confidence']}%")

        if gemini_result['evidence']:
            print(f"🔍 SUPPORTING EVIDENCE:")
            for evidence in gemini_result['evidence'][:3]:  # Show top 3
                print(f"   • {evidence}")

        # ==========================================================================
        # COMPARISON AND FINAL RECOMMENDATION
        # ==========================================================================

        print("\n📈 COMPARISON & FINAL ASSESSMENT")
        print("-" * 40)

        # Determine which analysis to trust more
        ml_score = ml_confidence if isinstance(ml_confidence, (int, float)) else 0
        gemini_score = gemini_result['confidence'] if isinstance(gemini_result['confidence'], (int, float)) else 0

        if ml_score >= 70 and gemini_score >= 70:
            if ml_diagnosis.lower() in gemini_result['diagnosis'].lower() or gemini_result['diagnosis'].lower() in ml_diagnosis.lower():
                print("✅ HIGH AGREEMENT: Both systems converge on similar diagnosis")
                final_diagnosis = gemini_result['diagnosis']  # Prefer LLM for clinical phrasing
                final_confidence = max(ml_score, gemini_score)
                source = "Both ML and LLM (High Agreement)"
            else:
                print("⚠️  DIVERGENT OPINIONS: Systems suggest different diagnoses")
                final_diagnosis = f"ML: {ml_diagnosis} | LLM: {gemini_result['diagnosis']}"
                final_confidence = (ml_score + gemini_score) / 2
                source = "Divergent Opinions"
        elif ml_score >= 60:
            print("📊 PREFERRING ML: Strong model confidence")
            final_diagnosis = ml_diagnosis
            final_confidence = ml_score
            source = "ML Model"
        elif gemini_score >= 60:
            print("🧠 PREFERRING LLM: Strong clinical reasoning"
            final_diagnosis = gemini_result['diagnosis']
            final_confidence = gemini_score
            source = "Gemini LLM"
        else:
            print("❓ INCONCLUSIVE: Both systems have low confidence")
            final_diagnosis = "Insufficient evidence for definitive diagnosis"
            final_confidence = max(ml_score, gemini_score)
            source = "Inconclusive"

        print(f"\n🎯 FINAL RECOMMENDATION:")
        print(f"   Diagnosis: {final_diagnosis}")
        print(f"   Confidence: {final_confidence:.1f}%")
        print(f"   Source: {source}")

        # ==========================================================================
        # ADDITIONAL CLINICAL INSIGHTS
        # ==========================================================================

        print("\n💡 ADDITIONAL CLINICAL INSIGHTS")
        print("-" * 40)

        # Medication evidence
        medication_evidence = get_evidence_from_medications(extracted_medications, med_db)
        if medication_evidence:
            print("💊 MEDICATION CORRELATIONS:")
            for disease, meds in medication_evidence.items():
                print(f"   • {', '.join(meds)} → supports {disease}")

        # Key findings
        context_findings = analyze_report_context(full_text, classified_info)
        if context_findings:
            print("\n🔬 KEY CLINICAL FINDINGS:")
            for finding in context_findings[:5]:
                print(f"   • {finding}")

        # Clinical interpretation
        print("\n📋 CLINICAL INTERPRETATION:")
        if extracted_symptoms and any(symptom in ['dyspnea', 'orthopnea', 'syncope'] for symptom in extracted_symptoms):
            print("   • Cardiac symptoms present with objective findings")
        if 'systolic dysfunction' in extracted_conditions and '30' in full_text:
            print("   • Severe systolic dysfunction requires comprehensive management")
        if extracted_medications:
            print("   • Medication profile consistent with cardiac diagnosis")

        # Show full LLM reasoning if available
        if gemini_result.get('reasoning') and len(gemini_result['reasoning']) > 100:
            print(f"\n🧠 LLM CLINICAL REASONING (summary):")
            print(f"   {gemini_result['reasoning'][:300]}...")

        # Clean up
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"\n🧹 Cleaned up temporary file: {image_path}")

    except FileNotFoundError as e:
        print(f"\n❌ ERROR: Required file not found: {e}")
    except Exception as e:
        print(f"\n❌ ANALYSIS ERROR: {e}")

# ==============================================================================
# SUPPORTING FUNCTIONS (You already have these)
# ==============================================================================

def create_clinical_context(full_text, classified_info):
    """Create clinical context from extracted data"""
    context_parts = []

    # Add quantitative findings
    text_lower = full_text.lower()
    if "30" in full_text and "ejection fraction" in text_lower:
        context_parts.append("Severely reduced LVEF (30%)")
    if "950" in full_text and "bnp" in text_lower:
        context_parts.append("Elevated BNP (950 pg/mL)")
    if "150/92" in full_text:
        context_parts.append("Hypertensive (150/92 mmHg)")

    # Add key findings
    if classified_info["Findings"]:
        context_parts.append(f"Key findings: {', '.join(classified_info['Findings'][:3])}")

    return ". ".join(context_parts) if context_parts else "Standard clinical presentation"

# ==============================================================================
# EXECUTION
# ==============================================================================

if __name__ == '__main__':
    print("DUAL-ANALYSIS CARDIOLOGY REPORT SYSTEM")
    print("ML Model + Gemini LLM Comparison")
    print("=" * 50)

    # Optional: Set your Gemini API key
    # GEMINI_API_KEY = "your_api_key_here"  # Get from https://aistudio.google.com/
    GEMINI_API_KEY = None  # Will use environment variable if set

    pdf_file = 'cardiology_report_sample.pdf'

    if os.path.exists(pdf_file):
        analyze_with_dual_systems(pdf_file, GEMINI_API_KEY)
    else:
        print(f"❌ PDF file '{pdf_file}' not found.")
        print("Please ensure the file is in the current directory.")

    print("\n" + "=" * 50)
    print("Dual-analysis complete!")

SyntaxError: '(' was never closed (ipython-input-62739815.py, line 279)

### prediction from 1st doctor


In [ ]:
# Step 1: Install the Google AI Python SDK
!pip install -q -U google-generativeai

In [ ]:
!pip install -q -U --upgrade google-generativeai

#### reports

In [ ]:
# A fake patient report you can use for testing
sample_report = """
PATIENT NAME: John Doe
DATE: 2025-10-04

DOCTOR'S NOTES:
Patient presents with a persistent cough, shortness of breath, and fatigue for the last 3 weeks. Reports occasional chest tightness. No fever.

PRESCRIPTION:
- Albuterol Inhaler, 2 puffs every 4-6 hours as needed for wheezing.
- Fluticasone Propionate (Flovent HFA), 2 puffs twice daily.

LAB RESULTS:
- Spirometry test shows FEV1/FVC ratio of 65% (indicative of obstruction).
"""

In [ ]:
sample_report_heart_attack = """
PATIENT NAME: Robert Miller
DATE: 2025-10-05

DOCTOR'S NOTES:
Patient presented to the ER with severe, crushing chest pain radiating to his left arm and jaw. Onset was approximately 2 hours ago. Patient is diaphoretic (sweating profusely), pale, and complains of nausea and extreme shortness of breath.

PRESCRIPTION (Administered in ER):
- Nitroglycerin sublingual tablet, 0.4mg
- Aspirin, 325mg, chewed
- Morphine Sulfate, 2mg IV push for pain
- Initiated Heparin drip for anticoagulation

LAB RESULTS (Stat):
- Troponin I level: 2.5 ng/mL (Critically High; Normal <0.04 ng/mL)
- CK-MB: 30 U/L (High; Normal 0-25 U/L)
- ECG: Shows ST-segment elevation in leads V1-V4, consistent with an anterior wall myocardial infarction.
"""

In [ ]:
sample_report_dka = """
PATIENT NAME: Maria Garcia
DATE: 2025-10-05

DOCTOR'S NOTES:
19-year-old female with a known history of Type 1 Diabetes, brought in by family due to increasing confusion and lethargy over the past 24 hours. Patient's breathing is deep and rapid (Kussmaul respirations). Breath has a distinct fruity odor. She reports severe abdominal pain and has vomited twice.

PRESCRIPTION (Treatment Plan):
- Start IV Normal Saline at 1 L/hr
- Begin Insulin infusion protocol
- Monitor electrolytes and blood glucose hourly

LAB RESULTS:
- Blood Glucose: 650 mg/dL (Severely High)
- Arterial Blood Gas (ABG): pH 7.1 (Acidic; Normal 7.35-7.45)
- Serum Bicarbonate: 8 mEq/L (Low; Normal 22-29)
- Ketones: Large amount detected in urine and serum.
"""

In [ ]:
sample_report_pe = """
PATIENT NAME: David Chen
DATE: 2025-10-05

DOCTOR'S NOTES:
Patient presents with a sudden onset of sharp, pleuritic chest pain and severe dyspnea (difficulty breathing). He recently returned from a 12-hour international flight. Patient is tachycardic (heart rate 120 bpm) and tachypneic (respiratory rate 28/min).

PRESCRIPTION:
- Lovenox (Enoxaparin) 1mg/kg subcutaneous, initiated immediately.
- Patient to be admitted for continued anticoagulation with Warfarin.

LAB RESULTS / IMAGING:
- D-dimer: 2500 ng/mL (Elevated; Normal <500 ng/mL)
- CT Angiogram of the Chest: Report confirms the presence of a large filling defect in the right main pulmonary artery, consistent with a pulmonary embolism.
- Oxygen Saturation: 88% on room air.
"""

#### code


In [ ]:

# Step 2: Import libraries and access your API key from secrets
import google.generativeai as genai
from google.colab import userdata # The library to access Colab secrets

# Get the API key from the secret manager
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

print("Successfully configured and making the prediction.") # A small check to confirm it's working

# Step 3: Define your synthetic (fake) medical data
# You can swap this out with other fake reports to test different conditions.

# Step 4: Craft your detailed prompt
# The more specific your prompt, the better the model's output.
prompt = f"""
Analyze the following patient report and act as a medical expert to predict the most likely condition the physician is treating.

Provide your answer in a clear, structured format as follows:
1.  **Predicted Condition:** [Name of the disease]
2.  **Reasoning:** [A concise explanation of why you chose this condition, citing specific evidence from the report's notes, prescriptions, and lab results.]

---
PATIENT REPORT:
{sample_report_heart_attack}
---
"""

# Step 5: Initialize the model and generate the content
try:
    model = genai.GenerativeModel('gemini-pro-latest') # Using the updated model name
    response = model.generate_content(prompt)

    # Step 6: Print the clean output from the model
    print("\n--- AI Analysis Result ---")
    print(response.text)
    print("--------------------------")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your API key is correct and has been added to Colab secrets properly.")

Successfully configured and making the prediction.

--- AI Analysis Result ---
Here is the analysis of the patient report:

1.  **Predicted Condition:** Acute Myocardial Infarction (Heart Attack), specifically an Anterior Wall ST-Elevation Myocardial Infarction (STEMI).

2.  **Reasoning:** The diagnosis is strongly supported by a combination of the patient's symptoms, specific lab results, and ECG findings.

    *   **Clinical Presentation:** The patient's report of "severe, crushing chest pain radiating to his left arm and jaw," accompanied by diaphoresis (profuse sweating), nausea, and shortness of breath, is the classic presentation of an acute myocardial infarction.
    *   **Cardiac Biomarkers:** The **critically high Troponin I level (2.5 ng/mL)** is the most definitive evidence. Troponin is a protein released exclusively from damaged heart muscle cells, and a level this elevated confirms myocardial necrosis (heart muscle death). The elevated CK-MB also supports this finding.
   

In [ ]:
# Diagnostic Code Cell
import google.generativeai as genai
from google.colab import userdata

try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)

    print("Successfully configured. Checking available models...\n")

    model_found = False
    for model in genai.list_models():
      if 'generateContent' in model.supported_generation_methods:
        print(model.name)
        if 'gemini-1.0-pro' in model.name:
            model_found = True

    if model_found:
        print("\n[SUCCESS] Your API key can see 'gemini-1.0-pro'. The main code should work.")
    else:
        print("\n[ERROR] Your API key is working, but it CANNOT see 'gemini-1.0-pro'. This confirms the issue is with the Google Cloud Project setup (API not enabled or billing not configured).")

except Exception as e:
    print(f"An error occurred during configuration: {e}")
    print("This likely means the API key itself is invalid or has been deleted.")

Successfully configured. Checking available models...

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-

### ocr integerated for the above (enables user to upload reports)
---



In [1]:
# Step 1: Install necessary libraries
# We need google-generativeai for Gemini and PyMuPDF to read PDF files
!pip install -q -U google-generativeai
!pip install -q PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.8 MB/s eta 0:00:00


model that works when structured and complete input is given

In [2]:
# Step 2: Import all required libraries
import google.generativeai as genai
from google.colab import userdata
from google.colab import files # This is for the upload button
import fitz  # This is the PyMuPDF library
import io

# Step 3: Configure the Gemini API
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring API: {e}")
    # Stop execution if the API key is not found
    raise SystemExit("Please ensure GOOGLE_API_KEY is set in Colab secrets.")

# Step 4: Define a function to extract text from a PDF
def extract_text_from_pdf(pdf_file_bytes):
    """Takes the byte content of a PDF file and returns its text."""
    try:
        # Open the PDF file from its byte content
        pdf_document = fitz.open(stream=pdf_file_bytes, filetype="pdf")
        text = ""
        # Iterate through each page and extract text
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        return f"Error reading PDF file: {e}"

# Step 5: Create the file upload button
print("\nPlease upload a medical report (PDF format):")
uploaded = files.upload()

# Step 6: Process the uploaded file and run the prediction
# The 'uploaded' variable will be a dictionary. We process the first file if it exists.
if uploaded:
    # Get the name and content of the first uploaded file
    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    print(f"\nProcessing file: {file_name}")

    # Extract text from the PDF
    extracted_text = extract_text_from_pdf(file_content)

    if "Error reading PDF file" in extracted_text:
        print(extracted_text)
    else:
        # Craft the prompt with the extracted text
        prompt = f"""
        Analyze the following patient report and act as a medical expert to predict the most likely condition the physician is treating.

        Provide your answer in a clear, structured format as follows:
        1.  **Predicted Condition:** [Name of the disease]
        2.  **Reasoning:** [A concise explanation of why you chose this condition, citing specific evidence from the report's notes, prescriptions, and lab results.]

        ---
        PATIENT REPORT:
        {extracted_text}
        ---
        """

        # Initialize the model and generate the content
        try:
            model = genai.GenerativeModel('gemini-pro-latest')
            response = model.generate_content(prompt)

            # Print the clean output from the model
            print("\n--- AI Analysis Result ---")
            print(response.text)
            print("--------------------------")

        except Exception as e:
            print(f"An error occurred during prediction: {e}")
else:
    print("No file was uploaded.")

Gemini API configured successfully.

Please upload a medical report (PDF format):


Saving report1.pdf to report1.pdf

Processing file: report1.pdf

--- AI Analysis Result ---
1.  **Predicted Condition:** Pulmonary Embolism (PE)

2.  **Reasoning:** The diagnosis is strongly supported by a combination of the patient's history, clinical presentation, lab results, and definitive imaging.

    *   **Clinical Presentation & Risk Factors:** The patient's sudden onset of sharp, pleuritic chest pain and severe dyspnea are classic symptoms of a PE. His recent 12-hour international flight is a major risk factor for developing a deep vein thrombosis (DVT), which can dislodge and travel to the lungs, causing a PE.
    *   **Vital Signs:** Tachycardia (fast heart rate) and tachypnea (rapid breathing), along with low oxygen saturation (88%), are physiological responses to a significant blockage in the pulmonary artery, as the body struggles to compensate for the lack of oxygenated blood.
    *   **Lab and Imaging Results:** The evidence is conclusive. The **elevated D-dimer** (2500

In [2]:
# Step 1: Install/update the library
!pip install -q -U google-generativeai

# Step 2: Import libraries and access your API key from secrets
import google.generativeai as genai
from google.colab import userdata

# Step 3: Configure the Gemini API
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=api_key)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring API: {e}")
    raise SystemExit("Please ensure GOOGLE_API_KEY is set in Colab secrets.")

# --- NEW: Define a reusable prediction function ---
def get_prediction(report_text):
    """Takes a string of medical text and returns the model's prediction."""
    prompt = f"""
    Analyze the following patient information and act as a medical expert to predict the most likely condition the physician is treating.

    Provide your answer in a clear, structured format as follows:
    1.  **Predicted Condition:** [Name of the disease]
    2.  **Reasoning:** [A concise explanation of why you chose this condition, citing specific evidence from the text provided.]

    ---
    PATIENT INFORMATION:
    {report_text}
    ---
    """
    try:
        model = genai.GenerativeModel('gemini-pro-latest')
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"An error occurred during prediction: {e}"

# --- SCENARIO 1: Prediction based ONLY on a prescription ---
prescription_only_data = """
PRESCRIPTION:
- Medication: Metformin 500mg
- Instructions: Take one tablet by mouth twice daily with meals.
- Refills: 3
"""

print("\n--- Prediction based on Prescription ONLY ---")
prediction1 = get_prediction(prescription_only_data)
print(prediction1)
print("------------------------------------------")


# --- SCENARIO 2: Prediction based ONLY on lab results ---
lab_results_only_data = """
LABORATORY RESULTS:
- Test: Fasting Blood Glucose
  - Result: 145 mg/dL
  - Flag: HIGH
  - Reference Range: 70-99 mg/dL
- Test: Hemoglobin A1c
  - Result: 7.5%
  - Flag: HIGH
  - Reference Range: < 5.7%
"""

print("\n--- Prediction based on Lab Results ONLY ---")
prediction2 = get_prediction(lab_results_only_data)
print(prediction2)
print("------------------------------------------")

Gemini API configured successfully.

--- Prediction based on Prescription ONLY ---
1.  **Predicted Condition:** Type 2 Diabetes Mellitus

2.  **Reasoning:** The prescribed medication, Metformin, is the first-line oral drug used to manage high blood sugar. The dosage (500mg twice daily) and instructions (take with meals to reduce gastrointestinal side effects) are a standard regimen for treating Type 2 Diabetes. The presence of three refills indicates the management of a chronic, long-term condition, which is characteristic of diabetes.
------------------------------------------

--- Prediction based on Lab Results ONLY ---
1.  **Predicted Condition:** Diabetes Mellitus (most likely Type 2)

2.  **Reasoning:** The diagnosis is based on two key laboratory results that both exceed the established diagnostic criteria for diabetes:
    *   **Elevated Fasting Blood Glucose:** The patient's result of 145 mg/dL is significantly higher than the normal reference range and surpasses the 126 mg/dL